In [2]:
import plotly as py
from plotly import graph_objs as go
import numpy as np
from stl import mesh
import plotly.express as px
import plotly.io as pio 
pio.renderers.default = 'notebook_connected'
colors = ['darkblue', 'teal']
def build_obstacles(obstacles): 
    res = []
    for ptn in obstacles:
        new_obstacle = (ptn[0]*10 - 1, ptn[1]*10 - 1, 0, ptn[0]*10 + 1, ptn[1]*10 + 1, 30) 
        res.append(new_obstacle)
    return np.array(res)

def stl2mesh3d(stl_mesh):
    p, q, r = stl_mesh.vectors.shape 
    vertices, ixr = np.unique(stl_mesh.vectors.reshape(p*q, r), return_inverse=True, axis=0)
    I = np.take(ixr, [3*k for k in range(p)])
    J = np.take(ixr, [3*k+1 for k in range(p)])
    K = np.take(ixr, [3*k+2 for k in range(p)])
    return vertices, I, J, K

class Plot(object):
    def __init__(self, filename,obs, start, goal):
        self.filename = filename + ".html"
        self.data_obs = None
        self.data_drone_start = None
        self.data_drone_goal = None
        title = "Environment"
        self.layout = go.Layout(paper_bgcolor='rgb(255,255,255)',title_text=title, 
                                title_x=0.5,font_color='black',  width=800,height=800,
                                scene_camera=dict(eye=dict(x=1.25, y=-1.25, z=1)),scene = dict(
                    xaxis = dict(
                         backgroundcolor="rgb(247, 121, 99 )",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="rgb(99, 216, 247 )",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),
                    zaxis = dict(
                        backgroundcolor="rgb(247, 227, 99)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),),
                    )          
        self.fig = None 
        self.frames = []
        self.plot_obstacles(obs)
        self.plot_drones_start(start)
        self.plot_drones_goal(goal)

    def plot_obstacles(self, O):
        obsts = []
        for O_i in O:
            obs = go.Mesh3d(
                x=[O_i[0], O_i[0], O_i[3], O_i[3], O_i[0], O_i[0], O_i[3], O_i[3]],
                y=[O_i[1], O_i[4], O_i[4], O_i[1], O_i[1], O_i[4], O_i[4], O_i[1]],
                z=[O_i[2], O_i[2], O_i[2], O_i[2], O_i[5], O_i[5], O_i[5], O_i[5]],
                i=[7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                j=[3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                k=[0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                color='gray',
                opacity=0.70
            )
            obsts.append(obs)
            self.data_obs=obsts

    def plot_drones_start(self, pos): 
        my_mesh = mesh.Mesh.from_file('quadcopter.stl')
        vertices, I, J, K = stl2mesh3d(my_mesh)
        x, y, z = vertices.T
        colorscale= [[0,'black'], [1, 'green']] 
        mesh3D = go.Mesh3d(x=x*5+pos[0]*10,y=y*5+pos[1]*10,z=z*5+pos[2]*10, 
                           i=I, j=J, k=K, flatshading=True,colorscale=colorscale,
                            intensity=z, name='Agent',showscale=False)
        self.data_drone_start=mesh3D
    def plot_drones_goal(self, pos): 
        my_mesh = mesh.Mesh.from_file('quadcopter.stl')
        vertices, I, J, K = stl2mesh3d(my_mesh)
        x, y, z = vertices.T
        colorscale= [[0,'blue'], [1, 'red']] 
        mesh3D = go.Mesh3d(x=x*5+pos[0]*10,y=y*5+pos[1]*10,z=z*5+pos[2]*10, 
                           i=I, j=J, k=K, flatshading=True,colorscale=colorscale, 
                           intensity=z, name='Goal',showscale=False)
        self.data_drone_goal= mesh3D
    def draw(self, auto_open=False):
        self.fig = go.Figure(data = self.data_obs, layout=self.layout)
        self.fig['layout']['title']['font'] = dict(size=30)
        self.fig.add_trace(self.data_drone_start)
        self.fig.add_trace(self.data_drone_goal)
        self.fig.data[6].update(lighting=dict(ambient= 0.18,diffuse= 1,fresnel=  .1,
                                specular= 1,roughness= .1,facenormalsepsilon=0))
        self.fig.data[6].update(lightposition=dict(x=300,y=300,z=100))
        self.fig.data[5].update(lighting=dict(ambient= 0.18,diffuse= 1,fresnel=  .1,
                                specular= 1,roughness= .1,facenormalsepsilon=0))
        self.fig.data[5].update(lightposition=dict(x=300,y=300,z=100))
        py.offline.plot(self.fig, filename=self.filename, auto_open=auto_open)
        self.fig.show()
    def animate(self, states): 

        self.fig = go.Figure(data = self.data_obs, layout=self.layout)
        self.fig.add_trace(self.data_drone_start)
        self.fig.add_trace(self.data_drone_goal)
        self.fig.update_layout(updatemenus=[dict(buttons = [dict(args = [None, {"frame": {"duration": 60, 
                  "redraw": True}, "fromcurrent": False, "transition": {"duration": 30}}],
                  label = "simulate", method = "animate")], type='buttons', showactive=True,y=1,
                  x=1.12, xanchor='right', yanchor='top')])
        frames = []
        for i in range(len(states)):
            self.plot_drones_start(states[i])
            data=[self.data_drone_start,self.data_drone_goal] 
            for j in range(len(self.data_obs)): 
                data.append(self.data_obs[j])
            f1 = go.Frame(data=data, name=f'{i}')
            frames.append(f1)

        self.fig['frames']=frames     
        self.fig.write_html("env.html",full_html=True, include_plotlyjs='cdn')

                  
        


In [25]:


history = np.load('state_history.npy')
init_stat = history[-1]
obstacles = [(-1.2, 2.0), (-1.2,0), (-0.6, 3.0), (2.5, 0), (1.6, -0.5)]
Obst = build_obstacles(obstacles)
line = np.linspace(0,0.93361929,200)
states = []
for i in line: 
    st = [i, i, i]
    states.append(st)
#states = [[1.0,2.0,2.0], [1.2,2.0,2.0],[1.4,2.0,2.0],[1.6,2.0,2.0],[1.8,2.0,2.0],[2.0,2.0,2.0],[2.0,1.8,2.0],[2.0,1.6,2.0]]
ploting = Plot("environment",Obst,[0.0,0.0,0.0],[0.93361929,0.93361929,0.93361929])
ploting.draw(auto_open=True)
#ploting.animate(states)


In [23]:
history = np.load('state_history.npy')
history[300]

array([[ 0.9072123 ,  0.9072123 ,  0.9072123 ],
       [ 0.97534395, -0.71072436,  0.25567743],
       [-1.540627  , -0.65087115,  1.29702898],
       [-1.57423052,  2.43824402,  0.4697359 ],
       [ 0.98981446, -2.14388209,  0.66602493],
       [ 0.74207576,  0.99833491,  1.25196328]])

In [21]:
history[-100]

array([[ 0.9072123 ,  0.9072123 ,  0.9072123 ],
       [ 0.97534395, -0.71072436,  0.25567743],
       [-1.540627  , -0.65087115,  1.29702898],
       [-1.57423052,  2.43824402,  0.4697359 ],
       [ 0.98981446, -2.14388209,  0.66602493],
       [ 0.74207576,  0.99833491,  1.25196328]])